# Шаг 1. Изучение данных

Импортируем библиотеки, необходимые для работы над проектом:

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats as st
import seaborn as sns

Вводим переменную для исходной анализируемой таблицы:

In [2]:
data = pd.read_csv('C:\games.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Значительное число пропусков обнаружено в столбцах с оценками и рейтингом игр (от 6 до 8 тыс. строк),также около 300 строк находится в столбце с информацией о годе выпуска.

Исходные данные содержат следующие столбцы:

- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы долларов)
- EU_sales — продажи в Европе (миллионы долларов)
- JP_sales — продажи в Японии (миллионы долларов)
- Other_sales — продажи в других странах (миллионы долларов)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

# Шаг 2. Предобработка данных

Мы привели названия столбцов к нижнему регистру

In [3]:
data.columns = data.columns.str.lower()

#### Обработка пропусков

Проанализируем столбцы на наличие пропусков

In [4]:
data.isnull().mean()

name               0.000120
platform           0.000000
year_of_release    0.016093
genre              0.000120
na_sales           0.000000
eu_sales           0.000000
jp_sales           0.000000
other_sales        0.000000
critic_score       0.513192
user_score         0.400897
rating             0.404786
dtype: float64

In [5]:
data.query('critic_score.isnull()').describe()
data.query('critic_score.isnull()').groupby('platform')['name'].count().sort_values(ascending=False).head(10)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
data.query('user_score.isnull()').describe()
data.query('user_score.isnull()').groupby('platform')['name'].count().sort_values(ascending=False).head(10)

In [ ]:
data.query('rating.isnull()').describe()
data.query('rating.isnull()').groupby('platform')['name'].count().sort_values(ascending=False).head(10)

Судя по всему пропуски в столбцах с информацией об оценках и рейтингах главным образом встречаются для старых игр, которым более 5-10 лет. Соответственно при взгляде на данные в разрезе платформ наибольшее количество пропусков относится к наиболее старшим поколениям

In [ ]:
data['year_of_release'].fillna(data.groupby('name')['year_of_release'].transform('median').round(), inplace = True)
data['year_of_release'].isnull().sum()

Оставшиеся строки не содержащие данные о годе выпуска мы удаляем

In [ ]:
data = data.loc[data['year_of_release'] != 0]

Аббервиатура "tbd" означает неопределенный статус, по причине недостаточности объема данных для выставления рейтинга. 
Заполнение пропусков и подобных значений не целесообразно и может существенно исказить выводы., поэтому нра текущий момент никаких изменений внесено не будет

In [ ]:
data['user_score'] = data['user_score'].replace('tbd', np.nan)

2 строки с пропуском имени и данных об оценке мы удалим 

In [ ]:
data.dropna(subset=['name'], inplace=True)

In [ ]:
data.isnull().mean()

Мы рассчитали суммарные продажи во всех регионах

В столбце рейтинг мы привели систему оценок к актуальной шкале

In [ ]:
data['rating'].unique()

In [ ]:
data['rating'] = data['rating'].replace('K-A', 'E')

In [ ]:
data['user_score'] = data['user_score'].astype('float32')

In [ ]:
data.describe()

Мы проанализировали данные на наличие необычных значений. Все столбцы в которых содержится числовая инфомрация соответствуют ожиданиям и пониманию

Совокупные данные о прадажах игр посчитаны в столбец total sales

In [ ]:
data['total_sales']= data['na_sales'] + data['eu_sales'] + data['jp_sales'] + data['other_sales']

In [ ]:
data.info()

##### В ходе предобработки были осуществлены следующие преобразования:
1. Названия колонок приведены к нижнему регистру
2. Более подробно изучены значения в каждом столбце на предмет наличия ошибочных значений.
3. Заполнена часть пустых значений в годах релиза в зависимости от года релиза на других платформах. Оставшаяся часть удалена.
4. Значения "tbd" в user_score объяснены и заменены на NaN.
5. Посчитаны суммарные продажи во всех регионах и сохранены в столбце sales_sum


# Шаг 3. Анализ данных

Для того, чтобы проанализировать какое количество игр выходило в разные годы построим график

In [ ]:
data.groupby('year_of_release')['name'].count().plot(figsize = (15, 4), grid = True, style = '-o')
plt.xlabel('Год')
plt.ylabel('Количество игр')
plt.title ('Количество выпущенных игр в разные годы')

Судя по графику на рост количества игр в начале 2000х могло повлиять технологическое развитие отрасли, спад в конце 2000х вероятнее всего обусловлен мировым финансовым кризисом 2008 года, в связи с которым финансирование новых проектов было сокращено.

Мы рассчитали сколько в среднем приносила каждая игра за год своего существования на рынке и показали совокупные показатели в разрезе платформ, жанров

In [ ]:
data['existing_period'] = 2017 - data['year_of_release']
data['revenue_per_year'] = data['total_sales'] / data['existing_period']

In [ ]:
data.pivot_table(index=['platform'], values='revenue_per_year', aggfunc='sum').sort_values(by='revenue_per_year', ascending=False).head(10)

In [ ]:
data.pivot_table(index=['genre'], values='revenue_per_year', aggfunc='sum').sort_values(by='revenue_per_year', ascending=False).head(10)

Как мы могли заметить, в рассматриваемом датасете наиболее прибыльными являются приставки производства компании Sony. Как мы видим на текущий момент наиболее финансово успешной консолью является PS4, ее идеологический конкурент XOne находится на 7 месте, и отстает более чем в 2 раза по уровню ежегодной выручки. Консоли предыдущего поколения занимают 2 и 3 места. PC является наименее прибыльной платформой на равне с первой выпущенной PS.

Данный анализ помогает понять историческую предрасположенность пользователей, где голос в пользу платформу или жанра был выражен не просто в качестве мнения, а был запечатлен монетой.

###  Анализ и определение наиболее актуальных платформ/периода для анализа

Мы визуализировали продажи по всем платформам по имеющемуся датасету

In [ ]:
all_platforms = ( data.pivot_table(index = 'year_of_release', columns = 'platform', 
                             values = 'total_sales', aggfunc = 'sum', fill_value = 0) )

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(all_platforms, linewidths=.1, ax=ax, cbar_kws= {'orientation': 'horizontal'}, vmax = 100)

Очевидно, что у каждой платформы есть свой жизненный цикл и уровень популярности за этот период. 
- Так к примеру PS - 6 лет, PS2 - 8 лет, PS3 - 7 лет. При этом переходный период пользователей на следующее поколение составляет в среднем 1-2 года и спрос на игры предыдущего поколения начинает стремительно падать, так скажем настает период распродажи остатков. 

- Наиболее успешной для Microsoft удалась X360, которая показала сопоставимые продажи с главными конкурентами и сохраняла их около 8 лет, XOne не удалось повторить успех своего предшественника.

- В аналогичный период успеха добилась компания Nintendo со своими DS и Wii однако их период активных продаж составил всего около 4-5 лет. Причем новое поколение именно портативной версии консоли, которая стала гибридом двух предыдущих стало пользоваться успехом. С учетом анонса выхода нового поколения на 2017 год, стоит проанализировать текущих представителей.

### Определение границ датасета для дальнейшего использования в анализе

В ходе анализа популярности платформ было определено, что популярностью пользуются главным образом игры, выпускаемые на консоли таких производителей как Sony, Microsoft и Nintendo. Мы определили жизненный цикл, сменяемость и относительную распространенность/популярность консолей. 

Наиболее актуальными являются PS4, XOne и 3DS. Однако для полноты картины необходимо учесть также информацию о предыдущем поколении, а именно PS3, X360 и DS/Wii.

Соответственно срез датасета мы будем осуществлять именно по таким параметрам без привязки ко времени выхода игр, т.к. для понимания предпочтений эти параметры будут первостепенными.

Геймплейные особенности, позиционирование консолей и фансообщество сформировало отличительные черты и преимущества консолей.

In [ ]:
top_platforms_list = ['Wii', 'DS', 'X360', 'PS3','PS4', '3DS', 'XOne']

In [ ]:
top_platforms = data.query('@top_platforms_list in platform')

In [ ]:
top_platforms_pivot = ( top_platforms.pivot_table(index = 'year_of_release', columns = 'platform', 
                             values = 'total_sales', aggfunc = 'sum', fill_value = 0) )

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(top_platforms_pivot, linewidths=.1, ax=ax, cbar_kws= {'orientation': 'horizontal'}, vmax = 100)

Актуальным периодом для планирования рекламы на 2017 год: 2005 год - настоящее время. Данный интервал позволит наиболее точно определить портрет пользователя каждой консоли и их жанровые предпочтения.

In [ ]:
df = top_platforms.query('year_of_release >= 2005')

#### Анализ взаимосвязи продаж и оценок игр в СМИ

### PS4

In [ ]:
df.query('platform == "PS4"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на PS4')

plt.show

df.query('platform == "PS4"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на PS4')

### PS3

In [ ]:
df.query('platform == "PS3"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на PS3')

plt.show

df.query('platform == "PS3"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на PS3')

### XOne

In [ ]:
df.query('platform == "XOne"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на XOne')

plt.show

df.query('platform == "XOne"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на XOne')

### X360

In [ ]:
df.query('platform == "X360"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на X360')

plt.show

df.query('platform == "X360"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на X360')

### Wii

In [ ]:
df.query('platform == "Wii"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на Wii')

plt.show

df.query('platform == "Wii"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на Wii')

### DS

In [ ]:
df.query('platform == "DS"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на DS')

plt.show

df.query('platform == "DS"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на DS')

### 3DS

In [ ]:
df.query('platform == "3DS"').plot(kind = 'scatter',
        x = 'user_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь пользовательского рейтинга и продаж на 3DS')

plt.show

df.query('platform == "3DS"').plot(kind = 'scatter',
        x = 'critic_score',
        y = 'total_sales',
        alpha = 0.5,
        figsize = (8, 4))
plt.title('Взаимосвязь рейтинга критиков и продаж на 3DS')

Зависимость выручки от мнения пользователей/критиков разнится  для каждого производителя приставок, но все же общая тенденция более успешных продаж при более высокой оценке сохраняется.

Оценим коэффициенты корреляции для наиболее актуальных платформ: Wii, PS4 и XOne

In [ ]:
df.query('platform == "PS4"')['user_score'].corr(df['total_sales'])

In [ ]:
df.query('platform == "PS4"')['critic_score'].corr(df['total_sales'])

In [ ]:
df.query('platform == "XOne"')['user_score'].corr(df['total_sales'])

In [ ]:
df.query('platform == "XOne"')['critic_score'].corr(df['total_sales'])

In [ ]:
df.query('platform == "Wii"')['user_score'].corr(df['total_sales'])

In [ ]:
df.query('platform == "Wii"')['critic_score'].corr(df['total_sales'])

Мы видим аналогичную ситуацию для PS4 и XOne: мнение критиков имеет умеренное влияние на объем продаж, мнение пользователей влияния не имеет.

Пользователи Wii в большей степени обращают внимание на отзывы пользователей, однако влияние критиков менее ощутимо, чем у других платформ. 

### Анализ выручки от продажи игр в разрезе платформ

In [ ]:
top_platforms_summury = ( df.groupby('platform')
                       .agg({'year_of_release': 'count', 'total_sales':'sum'})
                       .rename(columns = {'year_of_release': 'games'})
                       .sort_values(by = 'total_sales', ascending = False) )

top_platforms_summury['rev_per_game'] = top_platforms_summury['total_sales'] / top_platforms_summury['games']
top_platforms_summury
top_platforms_summury.plot(kind='bar')

In [ ]:
top_platforms_summury['rev_per_game'].plot(kind='bar')

В таблице выше представлены данные о выпущенном количестве игр, суммарной выручке и средней выручке на 1 выпущенную игру.
Как видно на графиках, средняя выручка на игру на PS3, X360 и Wii сопоставима. PS4 лидирует и на несколько пунктов обходит Xone, а DS значительно уступает в данном показателе, общий сопоставимый уровень выручки достигнут за счет значительно большего количества выпущенных игр.


In [ ]:
( df.boxplot(column=['total_sales'], by=['platform'], showfliers = False) )
plt.title(' ')

Данный график показывает нам, что наиболее прибыльные отдельные игры были выпущены на консоли Microsoft и Sony. Их более старшие модели уже соответствуют своим предшественникам. Игры на Nintendo в среднем менее прибыльны, но Wii наиболее успешен, судя по текущей визуализации. Медианные значения находятся на сопоставимых уровнях, но их положение относительно средних значений говорит о том, что на каждой из платформ есть игры, которые принесли напорядок больше выручки по сравнению с остальными играми.

### Анализ выручки от продажи игр в разрезе жанров

In [ ]:
df.query('["PS4", "PS3"] in platform ').groupby('genre')['total_sales'].sum().sort_values(ascending = False)

In [ ]:
df.query('["X360", "XOne"] in platform ').groupby('genre')['total_sales'].sum().sort_values(ascending = False)

In [ ]:
df.query('["3DS", "Wii", "DS"] in platform ').groupby('genre')['total_sales'].sum().sort_values(ascending = False)

Мы проанализировали доходность жанров для каждого производителя платформ:

- для Sony и Microsoft тройка наиболее прибыльных жанров одинакова, это Shooter, Action и Sports.
- для Nintendo разрыв в прибыльности не такой наглядный, в топ вошли Misc, Sports и Action, а также RP, Simulation, Platform, Racing и Puzzle (который не приносит прибыли на других платформах), а жанр Shooter значительно менее прибылен.

<div style="border:solid gold 2px; padding: 20px"> <h1 style="color:gold; margin-bottom:20px">Комментарий наставника</h1>

Жанры отлично проанализированы, но немного не хватает графики.

## Портрет покупателя для каждого региона продаж

Самые популярные платформы (топ-5). Опишите различия в долях продаж.
Самые популярные жанры (топ-5). Поясните разницу.
Влияет ли рейтинг ESRB на продажи в отдельном регионе?

In [ ]:
region_sales_by_platform = ( df.groupby('platform').agg({'na_sales':'sum', 'eu_sales':'sum', 'jp_sales':'sum', 'other_sales':'sum', 'total_sales':'sum', 'revenue_per_year':'sum'})
 .sort_values(by = 'revenue_per_year', ascending = False).reset_index() )

region_sales_by_platform['share_na'] = region_sales_by_platform['na_sales'] / region_sales_by_platform['total_sales']  
region_sales_by_platform['share_eu'] = region_sales_by_platform['eu_sales'] / region_sales_by_platform['total_sales']  
region_sales_by_platform['share_jp'] = region_sales_by_platform['jp_sales'] / region_sales_by_platform['total_sales'] 
region_sales_by_platform['share_other'] = region_sales_by_platform['other_sales'] / region_sales_by_platform['total_sales']  
region_sales_by_platform

In [ ]:
region_sales_by_platform.groupby('platform')['na_sales'].sum().plot.pie()
plt.title('Северная Америка')
plt.show()
region_sales_by_platform.groupby('platform')['eu_sales'].sum().plot.pie()
plt.title('Европа')
plt.show()
region_sales_by_platform.groupby('platform')['jp_sales'].sum().plot.pie()
plt.title('Япония')
plt.show()

По сводной таблице и графикам можно сделать следующие выводы:

- na и eu рынки очень схожи, они дают Sony 70% выручки, jp всего 5%
- консоли Microsoft наиболее популярны в na, что можно попробовать объяснить поддержкой своего производителя:), и что интересно, X360 пользовался в jp большей популярностью, чем PS3/4 однако в структуре выручки XOne jp составляет всего     
- консоли 3DS и DS имею более широкое распространение и наибольшие доли присутствия в jp по сравнению с прочими

В целом данная визуализация дает четкое представление о наиболее прибыльных платформах и их распределении по регионам.

In [ ]:
region_sales_by_genre = ( df.groupby('genre').agg({'na_sales':'sum', 'eu_sales':'sum', 'jp_sales':'sum', 'other_sales':'sum', 'total_sales':'sum', 'revenue_per_year':'sum'})
 .sort_values(by = 'revenue_per_year', ascending = False).reset_index() )

region_sales_by_genre['share_na'] = region_sales_by_genre['na_sales'] / region_sales_by_genre['total_sales']  
region_sales_by_genre['share_eu'] = region_sales_by_genre['eu_sales'] / region_sales_by_genre['total_sales']  
region_sales_by_genre['share_jp'] = region_sales_by_genre['jp_sales'] / region_sales_by_genre['total_sales'] 
region_sales_by_genre['share_other'] = region_sales_by_genre['other_sales'] / region_sales_by_genre['total_sales']  
region_sales_by_genre

In [ ]:
region_sales_by_genre.groupby('genre')['na_sales'].sum().plot.pie()
plt.title('Северная Америка')
plt.show()
region_sales_by_genre.groupby('genre')['eu_sales'].sum().plot.pie()
plt.title('Европа')
plt.show()
region_sales_by_genre.groupby('genre')['jp_sales'].sum().plot.pie()
plt.title('Япония')
plt.show()

Наиболее прибыльные жанры: Action, Shooter, Sports, RP и Misc. В разрезе жанров na и eu вновь показывают схожие результаты, а вот jp наиболее популярны RP игры, Strategy и Puzzle, что можно объяснить предрасположенностью к игре на Nintendo, где игры этих жанров представлены больше всего.

In [ ]:
rating_sales = ( df.groupby('rating').agg({'na_sales':'sum', 'eu_sales':'sum', 'jp_sales':'sum', 'total_sales':'sum'})
.sort_values(by = 'total_sales', ascending = False) )
rating_sales

In [ ]:
rating_sales.plot(kind = 'bar', figsize = (8, 6), grid = True)

В рассматриваемом периоде резонно анализировать следующие типы рейтингов: M (17лет+), E (0+), T (13+) и E10+ (10+). Как мы видим из графика, наиболее прибыльным являются игры, подходящие люям любого возраста, это можно обосновать тем, что потенциально данные игры разрабатываются для наиболее широкого круга людей, чаще всего ориентированы на детей или развлечения в компании. На втором месте идут игры с наиболее жестким рейтингом, как правило это трипл A проекты, у которых сформирована своя фан база, у них наивысшие бюджеты на разработку и соответствующий уровень качества. В целом влияние данных рейтингов весьма косвенно и условно, т.к. рейтинг в конечном счете является лишь рекомендацией, и не ограничивает к покупке, соответственно оценки игр и предрасположенность покупателя к самой игре/платформе.

##### Портрет NA пользователя 
Из актуальных платформ предпочитает играть на PS и XBox. Активнее покупателей из других регионов. Больше всего любит Shooter, Action, Sport, Misc и RP. 
##### Портрет EU пользователя 
Из актуальных платформ предпочитает играть на PS. Любимые жанры аналогичны NA. 
##### Портрет JP пользователя 
Предпочитает разнообразные игры на Nintendo, главным образом RP и Action. Если все же отдает предпочтение другой консоли, то это PS.

# Проверка гипотез

##### Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.

Период для сравнения рейтингов выберем 2014 - тек. год, (с момента выхода XOne)

In [ ]:
data.query('platform == "XOne" and year_of_release')['user_score'].agg({'count', 'mean'})

In [ ]:
data.query('platform == "PC" and year_of_release > 2013')['user_score'].agg({'count', 'mean'})

При сравнении за сопоставимый период средняя оценка пользователей выше у XOne.
- Н0: cредние пользовательские рейтинги платформ Xbox One и PC одинаковые. 
- Н1: cредние пользовательские рейтинги платформ Xbox One и PC не равны.

In [ ]:
alpha = .1
results1 = ( st.ttest_ind(data.query('platform == "PC" and year_of_release > 2013')['user_score'].dropna(), 
             data.query('platform == "XOne"')['user_score'].dropna(), equal_var = False) )

if (results1.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")
results1.pvalue

Вывод: По итогам проверки мы можем сделать вывод о том, что средний пользовательский рейтинг игр на XOne и PC с высокой вероятностью не отличается

#### Средние пользовательские рейтинги жанров Action и Sports разные.

In [ ]:
data.query('genre == "Action"')['user_score'].agg({'count', 'mean'})

In [ ]:
data.query('genre == "Sports"')['user_score'].agg({'count', 'mean'})

При сравнении за сопоставимый период средняя оценка пользователей выше у Action.
- H0: cредние пользовательские рейтинги Action и Sport игр одинаковые.
- H1: cредние пользовательские рейтинги Action и Sport игр не равны.

In [ ]:
results2 = (st.ttest_ind(df.query('genre == "Sports"')['user_score'].dropna(), 
             df.query('genre == "Action"')['user_score'].dropna(), equal_var = False))

if (results2.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")
results2.pvalue

Вывод: По итогам проверки мы можем сделать вывод о том, что средний пользовательский рейтинг Sport игр  с высокой вероятностью  ниже, чем Action игр.

# Общий вывод

- Наиболее ценный период для анализа данных - период с 2006 года, т.к. начиная с этого периода индустрия пережила пик своего развития, а также были сформированы основные предпочтения покупателей в отношении производителей платформ. 
- Наиболее востребованные жанры: Action, Shooter, Sport и RP.
- Предпочтения покупателей, как жанровые, так и при выборе платформы отличаются в большей степени отличия видны между JP и прочими регионами.
- Возрастной рейтинг ESRB и рейтинг пользователей не является определяющим продажи фактором. А вот рейтинг критиков важен для продаж во всех регионах.
- Была осуществлена проверка гипотез, описанных в задании, в результате чего можно сделать вывод о том, что рейтинг PC и XOne игр с высокой вероятностью не отличается, а вот рейтинг Sport игр ниже, чем Action